In [1]:
#目的：手入力した文字を感情分析する

In [2]:
# 単語感情極性表の文字化け解消＞成功
import csv
filename = "pn_ja.dic.txt"

with open('pn_ja_dic.csv', 'w') as f:

    writer = csv.writer(f)
    writer.writerow(["単語", "読み方", "品詞", "単語感情極性値", ])

    with open(filename, "r",encoding='cp932') as f:
        line = f.readline()
        while line:
            line = line.split(":")
            line[3] = line[3].replace('\n','')

            #CSV出力
            writer.writerow(line)

            line = f.readline()

In [3]:
#csvファイルから特定の情報を取り出す＞成功

#まずpandasにする
import pandas as pd
pn_data=pd.read_csv('pn_ja_dic.csv')

print(pn_data)
print(pn_data.loc[3,['単語感情極性値']])

         単語   読み方   品詞   単語感情極性値
0       優れる  すぐれる   動詞  1.000000
1        良い    よい  形容詞  0.999995
2        喜ぶ  よろこぶ   動詞  0.999979
3       褒める   ほめる   動詞  0.999979
4      めでたい  めでたい  形容詞  0.999645
...     ...   ...  ...       ...
55120    ない    ない  助動詞 -0.999997
55121    酷い   ひどい  形容詞 -0.999997
55122    病気  びょうき   名詞 -0.999998
55123    死ぬ    しぬ   動詞 -0.999999
55124    悪い   わるい  形容詞 -1.000000

[55125 rows x 4 columns]
単語感情極性値    0.999979
Name: 3, dtype: object


In [4]:
#mecabを使って形態素に分け、原型を抽出する

import MeCab
def word_stem(text):
    tagger = MeCab.Tagger("-Ochasen")#形態素解析"-Ochasen"は形式を決めるパラメータ
    result_vector = tagger.parse(text)
    return result_vector

In [5]:
text="優れた作品だと認められたね。めでたいなあ。"

In [6]:
#コピペ：mecabの出力をリストにする
def mecab_list(text):
    tagger = MeCab.Tagger("-Ochasen")
    tagger.parse('')
    node = tagger.parseToNode(text)
    word_class = []
    while node:
        word = node.surface
        wclass = node.feature.split(',')
        if wclass[0] != u'BOS/EOS':
            if wclass[6] == None:
                word_class.append([word,wclass[0],wclass[1],wclass[2],wclass[5],""])
            else:
                word_class.append([word,wclass[0],wclass[1],wclass[2],wclass[5],wclass[6]])
        node = node.next
    return word_class

In [7]:
#文章の平均の感情極性を出力
result=[]

for stem in range(len(mecab_list(text))):
    for i in range(55125):
        if pn_data.at[i,'単語']==mecab_list(text)[stem][5]:
            result.append(pn_data.at[i,'単語感情極性値'])
            print(result)
print(sum(result)/len(result))

[1.0]
[1.0, -0.449062]
[1.0, -0.449062, -0.21579299999999998]
[1.0, -0.449062, -0.21579299999999998, -0.19584000000000001]
[1.0, -0.449062, -0.21579299999999998, -0.19584000000000001, -0.547493]
[1.0, -0.449062, -0.21579299999999998, -0.19584000000000001, -0.547493, 0.9996450000000001]
0.09857616666666667
